## Processing danych tabelarycznych

In [1]:
import numpy as np 
import pandas as pd 
import pydicom,os,cv2
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

### Siim_train_test

Najpierw tworzę ramke danych zapisanych w dicomie:

In [2]:
df = pd.read_csv('C:/Users/maaja/Desktop/AndrzejczukMaja/archive/siim/train-rle.csv')
train_fns = sorted(glob('C:/Users/maaja/Desktop/AndrzejczukMaja/archive/siim/dicom-images-train/*/*/*.dcm'))
test_fns = sorted(glob('C:/Users/maaja/Desktop/AndrzejczukMaja/archive/siim/dicom-images-test/*/*/*.dcm'))

In [3]:
missing = 0
multiple = 0
patients_data = []
for k,paths in enumerate(train_fns):
    patient = {}
    img_id = paths.split('\\')[-1]
    data = pydicom.dcmread(paths)
    try:
        tmp = df[df['ImageId'] == '.'.join(img_id.split('.')[:-1])]
        
        if tmp.shape[0] > 1: 
            multiple += 1
        rle = tmp[' EncodedPixels'].values
        if rle[0] == '-1':
            pixels = rle[0]
        else:    
            pixels = [i for i in rle]
        
        patient["UID"] = data.SOPInstanceUID
        patient['EncodedPixels'] = pixels
        patient["Age"] = data.PatientAge
        patient["Sex"] = data.PatientSex
        patient["Modality"] = data.Modality
        patient["BodyPart"] = data.BodyPartExamined
        patient["ViewPosition"] = data.ViewPosition
        patient["filepath"] = paths
        patients_data.append(patient)
    except:
        missing += 1


In [4]:
df_patients = pd.DataFrame(patients_data, columns=["UID", "EncodedPixels", "Age", 
                            "Sex", "Modality", "BodyPart", "ViewPosition"])

df_patients['Pneumothorax'] = df_patients['EncodedPixels'].apply(lambda x:0 if x == '-1' else 1)
df_patients['Pneumothorax'] = df_patients['Pneumothorax'].astype('int')
df_patients.head() 

,UID,EncodedPixels,Age,Sex,Modality,BodyPart,ViewPosition,Pneumothorax
0,1.2.276.0.7230010.3.1.4.8323329.1000.151787516...,-1,38,M,CR,CHEST,PA,0
1,1.2.276.0.7230010.3.1.4.8323329.10000.15178752...,-1,10,F,CR,CHEST,AP,0
2,1.2.276.0.7230010.3.1.4.8323329.10001.15178752...,-1,50,F,CR,CHEST,AP,0
3,1.2.276.0.7230010.3.1.4.8323329.10002.15178752...,-1,68,F,CR,CHEST,AP,0
4,1.2.276.0.7230010.3.1.4.8323329.10003.15178752...,-1,65,M,CR,CHEST,AP,0


In [5]:
df_patients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12047 entries, 0 to 12046
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   UID            12047 non-null  object
 1   EncodedPixels  12047 non-null  object
 2   Age            12047 non-null  object
 3   Sex            12047 non-null  object
 4   Modality       12047 non-null  object
 5   BodyPart       12047 non-null  object
 6   ViewPosition   12047 non-null  object
 7   Pneumothorax   12047 non-null  int32 
dtypes: int32(1), object(7)
memory usage: 706.0+ KB


Nie mamy wartości null

In [6]:
df_patients['Age'] = df_patients['Age'].astype('int') 

In [7]:
df_patients[df_patients['Age'] > 100]

,UID,EncodedPixels,Age,Sex,Modality,BodyPart,ViewPosition,Pneumothorax
1204,1.2.276.0.7230010.3.1.4.8323329.11106.15178752...,-1,413,F,CR,CHEST,AP,0
9624,1.2.276.0.7230010.3.1.4.8323329.4830.151787518...,-1,148,M,CR,CHEST,PA,0


Usuwamy niechcianych pacjentów

In [8]:
df_patients = df_patients.drop(df_patients[df_patients['Age'] > 100].index)
print(df_patients.shape)

(12045, 8)


In [9]:
df_patients['BodyPart'].value_counts()

CHEST    12045
Name: BodyPart, dtype: int64

#### Kodowanie

Na początku pozbywam się kolumn, które nie dają żadnych infromacji - kolumna Modality jedynie przyjmuje wartość CR, a kolumna BodyPart wartość CHEST

In [10]:
df_patients.drop(columns=['Modality', 'BodyPart', 'UID', 'EncodedPixels'], inplace=True)

Kodowanie wieku

In [11]:
def zmienwiek(row):
    if row['Age'] < 21:
        result =  "Young"
    elif row['Age'] < 45:
        result = "Middle_Adults" 
    elif row['Age'] < 60:
        result = "Adults" 
    else:
        result =  "Elderly"
    return result

df_patients.Age = df_patients.apply(zmienwiek, axis = 1)
df_patients.Age.value_counts()

Adults           4192
Middle_Adults    3918
Elderly          3009
Young             926
Name: Age, dtype: int64

Kodowanie na danych kategorycznych

In [12]:
df_patients['Sex'] = df_patients['Sex'].apply(lambda x: 0 if x == 'M' else 1)
df_patients['ViewPosition'] = df_patients['ViewPosition'].apply(lambda x: 0 if x == 'PA' else 1)

In [13]:
df_patients = pd.get_dummies(df_patients, columns=["Age"])

In [14]:
df_patients.head()

,Sex,ViewPosition,Pneumothorax,Age_Adults,Age_Elderly,Age_Middle_Adults,Age_Young
0,0,0,0,0,0,1,0
1,1,1,0,0,0,0,1
2,1,1,0,1,0,0,0
3,1,1,0,0,1,0,0
4,0,1,0,0,1,0,0


One Hot Encoding umożliwia bardziej ekspresyjną reprezentację danych kategorycznych. Wiele algorytmów uczenia maszynowego nie może bezpośrednio pracować z danymi kategorycznymi. Kategorie należy przekonwertować na liczby.

### siim

In [15]:
data = pd.read_csv("C:\\Users\\maaja\\Documents\\WB_Prepro\\metadata_siim.csv")

In [16]:
data = pd.DataFrame(data, columns=["SOPInstanceUID","AccessionNumber",
                                     "BodyPartExamined","DeidentificationMethod","ImageType",
                                     "Modality","PatientID","PatientName","PatientSex","PixelRepresentation",
                                     "SeriesInstanceUID","StudyDate","StudyInstanceUID","StudyTime"])

In [17]:
data.head()

,SOPInstanceUID,AccessionNumber,BodyPartExamined,DeidentificationMethod,ImageType,Modality,PatientID,PatientName,PatientSex,PixelRepresentation,SeriesInstanceUID,StudyDate,StudyInstanceUID,StudyTime
0,000a312787f2,f6cd7f2019f5,CHEST,CTP Default: based on DICOM PS3.15 AnnexE. De...,"('ORIGINAL', 'PRIMARY')",DX,dcc3fdf6426a,d4268f26ed05,M,0,81456c9c5423,526316e376d4,5776db0cec75,8ccfb6acf07f
1,000c3a3f293f,1c2708371bc6,CHEST,CTP Default: based on DICOM PS3.15 AnnexE. De...,"('ORIGINAL', 'PRIMARY')",CR,f09ff9b7dab3,ef8c31f8dfdd,M,0,d8a644cc4f93,d09eda152722,ff0879eb20ed,543adb46f494
2,0012ff7358bc,99428f65d948,PORT CHEST,CTP Default: based on DICOM PS3.15 AnnexE. De...,"('DERIVED', 'PRIMARY')",DX,6a02c0731039,2f2fb944fe44,F,0,22897cd1daa0,96fc21dd2b1f,9d514ce429a7,746186a2bbf2
3,001398f4ff4f,7b889627faf1,CHEST,CTP Default: based on DICOM PS3.15 AnnexE. De...,"('DERIVED', 'PRIMARY', 'POST_PROCESSED', 'RT',...",CR,db63d9b7f61c,9ccd87b4511b,F,0,4d47bc042ee6,3f094e1700ec,28dddc8559b2,9c4f53a2d5e4
4,001bd15d1891,e9d93edf50c6,CHEST,CTP Default: based on DICOM PS3.15 AnnexE. De...,"('ORIGINAL', 'PRIMARY')",DX,605693f36f43,35bf9ddbf9e9,M,0,49170afa4f27,a69d89c8a474,dfd9fdd85a3e,5399ff949226


Zauważamy, że możemy usunac wszystkie kolumny zakodowane

In [18]:
data.drop(columns=['SOPInstanceUID', 'AccessionNumber', 'PatientID', 'PatientName',
                          'SeriesInstanceUID', 'StudyDate', 'StudyInstanceUID', 'StudyTime'], inplace=True)

In [19]:
data.head()

,BodyPartExamined,DeidentificationMethod,ImageType,Modality,PatientSex,PixelRepresentation
0,CHEST,CTP Default: based on DICOM PS3.15 AnnexE. De...,"('ORIGINAL', 'PRIMARY')",DX,M,0
1,CHEST,CTP Default: based on DICOM PS3.15 AnnexE. De...,"('ORIGINAL', 'PRIMARY')",CR,M,0
2,PORT CHEST,CTP Default: based on DICOM PS3.15 AnnexE. De...,"('DERIVED', 'PRIMARY')",DX,F,0
3,CHEST,CTP Default: based on DICOM PS3.15 AnnexE. De...,"('DERIVED', 'PRIMARY', 'POST_PROCESSED', 'RT',...",CR,F,0
4,CHEST,CTP Default: based on DICOM PS3.15 AnnexE. De...,"('ORIGINAL', 'PRIMARY')",DX,M,0


In [20]:
data.BodyPartExamined.value_counts()

CHEST         5047
TORAX          307
PORT CHEST     249
T?RAX          156
THORAX          77
SKULL           57
Pecho           21
ABDOMEN         21
2- TORAX         7
TÒRAX            5
PECHO            3
Name: BodyPartExamined, dtype: int64

Widzimy, że mamy wartości 'T?ORAX', 'TORAX' oraz 'TÓRAX' '2-  TORAX', ktore jak domyslamy sie oznaczaja to samo, ponieważ 'TÓRAX' to hiszpańskie słowo na angielskie 'THORAX'. Jednak dodatkowo możemy zauważyć, że CHEST i THORAX to są synonimy dlatego te grupy zdjęć można połączyć. 

In [21]:
data = data.replace('TORAX','CHEST').replace('T?RAX','CHEST').replace('TÒRAX','CHEST').replace('THORAX','CHEST').replace('2- TORAX','CHEST')

Następnie usuwam te wiersze które nie dotyczą klatki piersiowej - nie będziemy brać ich pod uwagę. Zostawiamy podział na CHEST i PORT CHEST, zauważając że PORT CHEST jest to badania podczas którego jest na zdjęciu jest widoczna rurka przymocowana do ludzkiej żyły

In [22]:
data = data[(data['BodyPartExamined'] == "CHEST") | (data['BodyPartExamined'] == "PORT CHEST")]

Po większym zapoznaniu z kolumnami niestety stwierdziliśmy, że w tej ramce kolumny nie niosą za sobą wystarczająco dużo informacji aby wytrenować model, dlatego nie przystąpiliśmy do kodowania.